In [1]:
from __future__ import unicode_literals
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect('biosource')

In [2]:
import pandas as pd
df = pd.DataFrame(list(session.execute('SELECT * FROM pubmed')))

In [16]:
df

,pmid,abstractText,author_json,chemicals,date,geneSymbol,grants,journal_json,keywords,mesh,mesh_id,pubType,title,xml
0,1535,The syntheses of trans- and cis-1-benzyl-3-dim...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Ahmed"",""...","[Dimethylamines, Histamine H1 Antagonists, Pip...",None,None,None,"{""MedlineJournalInfo"":{""Country"":""United State...",None,"[Acetylcholine, Animals, Barium, Dimethylamine...",None,"[Journal Article, Research Support, U.S. Gov't...",Conformationally restricted analogs of histami...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
1,82694,Mathematical smoothing of data for the Framing...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Anderson...",None,None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Blood Pressure, Blood Pressure Determination,...",None,[Journal Article],Re-examination of some of the Framingham blood...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
2,57379,,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Visner"",...","[Phenytoin, Phenobarbital]",None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Abnormalities, Drug-Induced, Epilepsy, Female...",None,[Journal Article],Letter: Anticonvulsants and fetal malformations.,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
3,12775,A new technique is described for the measureme...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Wakeham""...","[Lipoproteins, Serum Albumin, Bovine]",None,None,None,"{""MedlineJournalInfo"":{""Country"":""Ireland"",""Me...",None,"[Animals, Blood, Cattle, Diffusion, Hydrogen-I...",None,[Journal Article],Diffusion coefficients for protein molecules i...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
4,76367,The sera of Heterakis-infected birds influence...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Stomenov...",[gamma-Globulins],None,None,None,"{""MedlineJournalInfo"":{""Country"":""Bulgaria"",""M...",None,"[Age Factors, Animals, Chickens, Immunization,...",None,"[English Abstract, Journal Article]",[Passive immunization in heterakidosis].,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
5,46073,,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Moffat"",...",None,None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Basement Membrane, Chronic Disease, Humans, K...",None,[Journal Article],Letter: Urinary reflux and chronic pyelonephri...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
6,89864,To examine the effect of amino acid substituti...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Ibrahimi...","[Epitopes, Peptide Fragments, Muramidase]",None,None,None,"{""MedlineJournalInfo"":{""Country"":""United State...",None,"[Amino Acid Sequence, Animals, Biological Evol...",None,"[Comparative Study, Journal Article, Research ...",Amino acid sequence of California quail lysozy...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
7,23958,Hemorrhagic erosions of the gastric mucosa wer...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Zabrodin...","[Autonomic Agents, Ganglionic Blockers, Parasy...",None,None,None,"{""MedlineJournalInfo"":{""Country"":""Russia (Fede...",None,"[Animals, Autonomic Agents, Electric Stimulati...",None,"[English Abstract, Journal Article]",[Effect of pharmacological substances on the d...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
8,48024,,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Hume"",""F...",None,None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Circadian Rhythm, Humans, Leukocyte Count, Ma...",None,[Journal Article],Letter: Arneth-Cooke count: timing cancer ther...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St..."
9,88647,,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Horsey"",...",[Bicarbonates],None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Bicarbonates, Humans, Methods, Terminology as...",None,[Letter],Actual or standard bicarbonate.,"<PubmedArticle><MedlineCi

In [12]:
from __future__ import unicode_literals, print_function, division
import spacy, sklearn, PySysrev, random, sys

GENE_DATA   = PySysrev.processAnnotations(project_id=3144, label='GENE')
train, test = GENE_DATA[:int(0.8 * len(GENE_DATA))], GENE_DATA[-int(0.2 * len(GENE_DATA)):]
print("{} train instances {} test instances".format(len(train),len(test)))

984 train instances 246 test instances


In [10]:
# create nlp model
nlp = spacy.blank('en')
ner = nlp.create_pipe('ner')
ner.add_label('GENE')
nlp.add_pipe(ner)
optimizer = nlp.begin_training()

# train model
epochs = 30
for itn in range(epochs):
    sys.stdout.write("{} ".format(itn))
    text, annotations = zip(*train) #unzip text/annotations
    nlp.update(text, annotations, sgd=optimizer, drop=0.2,losses=losses)

print("done!")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 done!


In [12]:
# creates a spacy_io 'Scorer' object with accuracy metrics
def evaluate(data):
    scorer = spacy.scorer.Scorer()
    for text,entity_offsets in data: 
        gold_value    = spacy.gold.GoldParse(nlp.make_doc(text),entities=entity_offsets.get('entities'))
        pred_value    = nlp(text)
        try: 
            scorer.score(pred_value,gold_value)
        except ValueError:
            pass; # spacy has rare error w/ score fn - https://github.com/explosion/spaCy/issues/2661
    return scorer

# build evaluations
tests_eval, train_eval = evaluate(test).ner, evaluate(train).ner
mat = [["","TP","FN","FP","Recall","Precision"],
       ["TEST", tests_eval.tp, tests_eval.fn, tests_eval.fp, tests_eval.recall, tests_eval.precision],
       ["TRAIN",train_eval.tp, train_eval.fn, train_eval.fp, train_eval.recall, train_eval.precision]]

from IPython.display import HTML, display
import tabulate
display(HTML("<div style='margin: auto;width:50%'>{}</div>".format(tabulate.tabulate(mat,tablefmt='html'))))

,TP,FN,FP,Recall,Precision
TEST,262,116,176,0.693121693122,0.598173515982
TRAIN,1538,141,342,0.916021441334,0.818085106383


In [13]:
# now train with whole model and save it
# create nlp model
nlp = spacy.blank('en')
ner = nlp.create_pipe('ner')
ner.add_label('GENE')
nlp.add_pipe(ner)
optimizer = nlp.begin_training()

# train model
epochs = 30
for itn in range(epochs):
    losses = {}
    sys.stdout.write("{} ".format(itn))
    text, annotations = zip(*GENE_DATA) #unzip text/annotations
    nlp.update(text, annotations, sgd=optimizer, drop=0.2,losses=losses)

print("done!")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 done!


In [14]:
nlp.to_disk('/home/nolelin/gene_model')

In [4]:
import spacy

nlp = spacy.load('/home/nolelin/gene_model')

In [25]:
ner_prediction = [nlp(unicode(x)).ents for x in list(df['abstractText'])]
ner_prediction

[(),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (p50,),
 (),
 (),
 (),
 (),
 (),
 (HBsAg,),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (EDTA,),
 (GABA,),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (PAS, PAF),
 (),
 (),
 (),
 (DNP-IgG,),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (DTIC,),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (Paco2,),
 (PCO2,),
 (MA1,),
 (),
 (),
 (),
 (),
 (x-rays,),
 (),
 (amiA-r19, amiA-r23, amiA-r17),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (IgG,),
 (),
 (),
 (),
 (),
 (KBH4, KBH4),
 (),
 (),
 (),
 (),
 (),
 (),


In [26]:
df['NER_Genes'] = ner_prediction

In [27]:
df

,pmid,abstractText,author_json,chemicals,date,geneSymbol,grants,journal_json,keywords,mesh,mesh_id,pubType,title,xml,NER_Genes
0,1535,The syntheses of trans- and cis-1-benzyl-3-dim...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Ahmed"",""...","[Dimethylamines, Histamine H1 Antagonists, Pip...",None,None,None,"{""MedlineJournalInfo"":{""Country"":""United State...",None,"[Acetylcholine, Animals, Barium, Dimethylamine...",None,"[Journal Article, Research Support, U.S. Gov't...",Conformationally restricted analogs of histami...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
1,82694,Mathematical smoothing of data for the Framing...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Anderson...",None,None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Blood Pressure, Blood Pressure Determination,...",None,[Journal Article],Re-examination of some of the Framingham blood...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
2,57379,,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Visner"",...","[Phenytoin, Phenobarbital]",None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Abnormalities, Drug-Induced, Epilepsy, Female...",None,[Journal Article],Letter: Anticonvulsants and fetal malformations.,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
3,12775,A new technique is described for the measureme...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Wakeham""...","[Lipoproteins, Serum Albumin, Bovine]",None,None,None,"{""MedlineJournalInfo"":{""Country"":""Ireland"",""Me...",None,"[Animals, Blood, Cattle, Diffusion, Hydrogen-I...",None,[Journal Article],Diffusion coefficients for protein molecules i...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
4,76367,The sera of Heterakis-infected birds influence...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Stomenov...",[gamma-Globulins],None,None,None,"{""MedlineJournalInfo"":{""Country"":""Bulgaria"",""M...",None,"[Age Factors, Animals, Chickens, Immunization,...",None,"[English Abstract, Journal Article]",[Passive immunization in heterakidosis].,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
5,46073,,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Moffat"",...",None,None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Basement Membrane, Chronic Disease, Humans, K...",None,[Journal Article],Letter: Urinary reflux and chronic pyelonephri...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
6,89864,To examine the effect of amino acid substituti...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Ibrahimi...","[Epitopes, Peptide Fragments, Muramidase]",None,None,None,"{""MedlineJournalInfo"":{""Country"":""United State...",None,"[Amino Acid Sequence, Animals, Biological Evol...",None,"[Comparative Study, Journal Article, Research ...",Amino acid sequence of California quail lysozy...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
7,23958,Hemorrhagic erosions of the gastric mucosa wer...,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Zabrodin...","[Autonomic Agents, Ganglionic Blockers, Parasy...",None,None,None,"{""MedlineJournalInfo"":{""Country"":""Russia (Fede...",None,"[Animals, Autonomic Agents, Electric Stimulati...",None,"[English Abstract, Journal Article]",[Effect of pharmacological substances on the d...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
8,48024,,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Hume"",""F...",None,None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Circadian Rhythm, Humans, Leukocyte Count, Ma...",None,[Journal Article],Letter: Arneth-Cooke count: timing cancer ther...,"<PubmedArticle><MedlineCitation Owner=""NLM"" St...",()
9,88647,,"[{""Author"":{""ValidYN"":""Y"",""LastName"":""Horsey"",...",[Bicarbonates],None,None,None,"{""MedlineJournalInfo"":{""Country"":""England"",""Me...",None,"[Bicarbonates, Humans, Methods, Terminology as...",None,[Letter],Actual or standard bi